# Prerequisites
Understanding in
- machine learning and deep learning
- python syntax
- python libraries: numpy, pandas, Pytorch

# Introduction
This notebook shows the study of the [CWRU Bearing Dataset](https://csegroups.case.edu/bearingdatacenter/home), which contains data of normal and fault bearings. Artificial defects of different diameters (0.007 ~ 0.028 Inches) are manufactured at different locations of the bearings: inner raceway(IR), outer raceway(OR) and ball(B) defects. 

Vibration data was recorded for motor loads of 0 to 3 hp (motor speed of 1797 to 1720 RPM) using accelerometers at the drive end (DE) and fan end (FE) and the data is stored as Matlab files. The sampling rate is 12 kHz and each Matlab file contains between ~120k to ~240k sample points. For more information please refer to the [website](https://csegroups.case.edu/bearingdatacenter/home).

This study focuses on the classification of the drive end bearing defects using only the signal data at **DE**. It is a **multiclass classification** problem. The input is the vibration signal data at DE and the output is the type of defects:
- 0 : Normal (N), 
- 1 : Fault at Ball (B),
- 2 : Fault at Inner Raceway (IR), 
- 3 : Fault at Outer Raceway (OR), 



# Import

In [1]:
# Data science libraries
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Pytorch
import torch
from torch import nn
from torch.nn import functional as F
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
from torch.nn.modules.loss import CrossEntropyLoss

# Others
from IPython.core.debugger import set_trace
from pathlib import Path

from helper import get_df_all, download
from train_helper import get_dataloader, fit, validate 
import nn_model
from data_urls import URLS

In [2]:
working_dir = Path('.')
DATA_PATH = Path("./Data")
save_model_path = working_dir / 'Model'
DE_path = DATA_PATH / '12k_DE'

for path in [DATA_PATH, save_model_path]:
    if not path.exists():
        path.mkdir(parents=True)

In [3]:
# # Uncomment this to download the 12k_DE data if needed
# for name, url in URLS["DE_12k"].items():
#     download(url, DE_path, name, suffix=".mat")

In [4]:
#### HYPERPARAMETERS ####
bs = 64
lr = 0.001
wd = 1e-5
betas=(0.99, 0.999)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
random_seed = 42

# Load Data and Preprocessing

In [5]:
df_all = get_df_all(DE_path, segment_length=500, normalize=True)
features = df_all.columns[2:]
target = 'label'

In [6]:
df_all.sample(5)

,label,filename,0,1,2,3,4,5,6,7,...,490,491,492,493,494,495,496,497,498,499
17802,3,OR021@6_3.mat,0.274785,0.018349,0.003093,0.212311,0.105523,-0.129121,-0.008530,0.181073,...,0.256624,0.516693,0.115693,-0.108780,0.142572,0.032151,-0.170528,-0.307827,-0.158905,0.005999
14772,3,OR014@6_2.mat,-0.675053,-0.069198,1.270061,-0.380097,-0.266814,-0.426249,0.396941,-0.063743,...,-0.448906,-1.651386,-0.475758,-0.178705,-0.233668,0.980140,0.290371,-0.067520,0.467429,0.532042
2967,1,B028_0.mat,1.755181,1.672121,-1.291409,-1.508854,0.660901,1.297566,-0.496850,-0.891779,...,-0.115635,0.524556,0.134330,-0.456887,-0.157165,0.446198,-0.090952,-0.251196,0.450507,0.045001
11473,3,OR007@12_1.mat,-0.472892,0.070833,0.700586,-0.234298,-0.696699,0.232136,0.534579,-0.238331,...,0.328245,0.084275,-0.606639,-0.120714,0.437797,-0.264542,-0.437271,0.423011,0.570872,-0.134156
2023,1,B021_0.mat,0.064694,0.384914,0.501358,-0.678848,-0.803782,0.706347,0.881013,-0.135444,...,-0.632755,-0.499330,0.819152,1.008373,0.696644,0.024666,-0.243397,0.159304,0.255128,-0.009296


In [7]:
df_all.shape

(17987, 502)

In [8]:
## Split the data into train and validation set
X_train, X_valid, y_train, y_valid = train_test_split(df_all[features], 
                                                      df_all[target], 
                                                      test_size=0.20, random_state=random_seed, shuffle=True
                                                     )

In [9]:
## Create DataLoader of train and validation set
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_valid = torch.tensor(X_valid.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_valid = torch.tensor(y_valid.values, dtype=torch.long)

train_ds = TensorDataset(X_train, y_train)
valid_ds = TensorDataset(X_valid, y_valid)
train_dl, valid_dl = get_dataloader(train_ds, valid_ds, bs)

# Training with Adams Optimizer

In [10]:
## Instantiate model, optimizer and loss function
model = nn_model.CNN_1D_2L(len(features))
model.to(device)
opt = optim.Adam(model.parameters(), lr=lr, betas=betas, weight_decay=wd)
loss_func = CrossEntropyLoss()

In [11]:
%%time
## Train
epochs = 20
model, metrics = fit(epochs, model, loss_func, opt, train_dl, valid_dl, train_metric=False)

EPOCH 	 Train Loss 	 Val Loss 	 Train Acc 	 Val Acc 	
0 	 0.00685 	 0.37846 	 0.00000 	0.85937 	
1 	 0.00220 	 0.21542 	 0.00000 	0.91579 	
2 	 0.00130 	 0.12809 	 0.00000 	0.94803 	
3 	 0.00093 	 0.09389 	 0.00000 	0.96304 	
4 	 0.00073 	 0.10653 	 0.00000 	0.95414 	
5 	 0.00055 	 0.10663 	 0.00000 	0.95942 	
6 	 0.00062 	 0.06465 	 0.00000 	0.97360 	
7 	 0.00036 	 0.03604 	 0.00000 	0.98833 	
8 	 0.00053 	 0.02805 	 0.00000 	0.99138 	
9 	 0.00040 	 0.04993 	 0.00000 	0.98193 	
10 	 0.00033 	 0.08187 	 0.00000 	0.96665 	
11 	 0.00035 	 0.05626 	 0.00000 	0.98193 	
12 	 0.00032 	 0.12104 	 0.00000 	0.95664 	
13 	 0.00045 	 0.02969 	 0.00000 	0.98916 	
14 	 0.00042 	 0.05943 	 0.00000 	0.98277 	
15 	 0.00027 	 0.11805 	 0.00000 	0.95525 	
16 	 0.00027 	 0.04056 	 0.00000 	0.98749 	
17 	 0.00026 	 0.13035 	 0.00000 	0.95914 	
18 	 0.00019 	 0.03276 	 0.00000 	0.98777 	
19 	 0.00028 	 0.13098 	 0.00000 	0.95831 	
Wall time: 24min 10s


In [12]:
%%time
## Train
epochs = 20
model, metrics = fit(epochs, model, loss_func, opt, train_dl, valid_dl, train_metric=False)

EPOCH 	 Train Loss 	 Val Loss 	 Train Acc 	 Val Acc 	
0 	 0.00020 	 0.04433 	 0.00000 	0.98527 	
1 	 0.00017 	 0.08623 	 0.00000 	0.96915 	
2 	 0.00017 	 0.02013 	 0.00000 	0.99277 	
3 	 0.00021 	 0.03686 	 0.00000 	0.98666 	
4 	 0.00021 	 0.01674 	 0.00000 	0.99389 	
5 	 0.00020 	 0.04552 	 0.00000 	0.98082 	
6 	 0.00025 	 0.04389 	 0.00000 	0.98694 	
7 	 0.00016 	 0.04647 	 0.00000 	0.98138 	
8 	 0.00012 	 0.06574 	 0.00000 	0.97415 	
9 	 0.00007 	 0.05641 	 0.00000 	0.98027 	
10 	 0.00010 	 0.03147 	 0.00000 	0.98722 	
11 	 0.00010 	 0.03779 	 0.00000 	0.98555 	
12 	 0.00010 	 0.01386 	 0.00000 	0.99500 	
13 	 0.00009 	 0.01199 	 0.00000 	0.99611 	
14 	 0.00013 	 0.02183 	 0.00000 	0.99361 	
15 	 0.00022 	 0.01676 	 0.00000 	0.99277 	
16 	 0.00025 	 0.01729 	 0.00000 	0.99555 	
17 	 0.00016 	 0.01913 	 0.00000 	0.99444 	
18 	 0.00008 	 0.01996 	 0.00000 	0.99305 	
19 	 0.00008 	 0.02320 	 0.00000 	0.99277 	
Wall time: 24min 19s


# Save trained model

In [11]:
torch.save(model.state_dict(), save_model_path / 'model.pth')

In [12]:
model2 = nn_model.CNN_1D_2L(len(features))

In [13]:
model2.load_state_dict(torch.load(save_model_path / 'model.pth'))
model2.eval()

CNN_1D_2L(
  (layer1): Sequential(
    (0): Conv1d(1, 64, kernel_size=(9,), stride=(1,), padding=(4,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5)
    (4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5)
    (4): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  )
  (linear1): Linear(in_features=16000, out_features=4, bias=True)
)

In [14]:
%%time
print(validate(model, valid_dl, loss_func))

(0.012104931890057517, 0.9952751528627015)
Wall time: 92.8 ms
